## Set up the dataframe.

Read the data into a Rdd.

In [ ]:
import findspark
findspark.init('/usr/local/bin/spark-2.0.1')

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("Word Count") \
    .getOrCreate()

In [2]:
sc = spark.sparkContext
trainRdd = sc.textFile('input/tr.csv').cache()
testRdd = sc.textFile('input/test.csv').cache()

In [3]:
from pyspark.sql.types import *

def makeDF(rdd, data_set='train'):
    #Save the row in the rdd that contains the column names, then delete it from the rdd.
    header = rdd.first()
    header_rdd = rdd.filter(lambda l: 'margin' in l)
    rdd_no_header = rdd.subtract(header_rdd)

    #Prepare the schema and rdd to build the dataframe. 
    #Species will be a string while all columns will be floats.
    #Note if data_set == 'test', the won't have a species column
    fields = [StructField(field_name, FloatType(), True) for field_name in header.split(",")]   
    if data_set == 'train':
        fields[1].dataType = StringType()
        rdd_split = rdd_no_header.map(lambda l: l.split(",")).map(lambda l: [float(x) if i != 1 else str(x) for i,x in enumerate(l)])
    else:
        rdd_split = rdd_no_header.map(lambda l: l.split(",")).map(lambda l: [float(x) for i,x in enumerate(l)])

    schema=StructType(fields)
    
    return spark.createDataFrame(rdd_split, schema).cache()
    

In [4]:
train_df = makeDF(trainRdd)
test_df = makeDF(testRdd, data_set='test')

In [5]:
train_df.select('species').show()

+--------------------+
|             species|
+--------------------+
|Magnolia_Salicifolia|
|Betula_Austrosine...|
|  Tilia_Platyphyllos|
|     Ilex_Aquifolium|
|         Acer_Pictum|
|  Quercus_Imbricaria|
|   Quercus_Agrifolia|
|    Acer_Saccharinum|
|Quercus_Semecarpi...|
|    Cornus_Chinensis|
|     Acer_Capillipes|
|     Quercus_Pontica|
|      Viburnum_Tinus|
|         Sorbus_Aria|
|Liriodendron_Tuli...|
|Quercus_Infectori...|
|      Quercus_Texana|
|Quercus_Phillyrae...|
|   Quercus_Agrifolia|
|   Cotinus_Coggygria|
+--------------------+
only showing top 20 rows



## Prepare the data for training.

Label encode the target column in the training set.

In [6]:
from pyspark.ml.feature import IndexToString, StringIndexer
stInd = StringIndexer(inputCol='species', outputCol='speciesEnc')
stIndModel = stInd.fit(train_df)
train_df_enc = stIndModel.transform(train_df).cache()
train_df_enc.select('species', 'speciesEnc').show()

+--------------------+----------+
|             species|speciesEnc|
+--------------------+----------+
|Magnolia_Salicifolia|      38.0|
|Betula_Austrosine...|       1.0|
|  Tilia_Platyphyllos|      87.0|
|     Ilex_Aquifolium|      22.0|
|         Acer_Pictum|      82.0|
|  Quercus_Imbricaria|      31.0|
|   Quercus_Agrifolia|      32.0|
|    Acer_Saccharinum|      79.0|
|Quercus_Semecarpi...|      28.0|
|    Cornus_Chinensis|      74.0|
|     Acer_Capillipes|      43.0|
|     Quercus_Pontica|      50.0|
|      Viburnum_Tinus|       8.0|
|         Sorbus_Aria|       0.0|
|Liriodendron_Tuli...|      73.0|
|Quercus_Infectori...|      23.0|
|      Quercus_Texana|      75.0|
|Quercus_Phillyrae...|      44.0|
|   Quercus_Agrifolia|      32.0|
|   Cotinus_Coggygria|      64.0|
+--------------------+----------+
only showing top 20 rows



Create the features column that will be used to train the model.

In [7]:
from pyspark.ml.feature import VectorAssembler
cols = train_df_enc.columns
cols.remove('species')
cols.remove('speciesEnc')
cols.remove('id')
va = VectorAssembler(inputCols=cols, outputCol="features")
test_df_vec = va.transform(test_df)
train_df_vec = va.transform(train_df_enc)



In [10]:
train_preds = model.transform(train_df_vec).cache()

In [11]:
train_preds = train_preds.select('species', 'speciesEnc', 'probability', 'prediction', 'id').cache()

In [12]:
ordered = train_preds.orderBy('speciesEnc')
ordered.show()

+--------------------+----------+--------------------+----------+------+
|             species|speciesEnc|         probability|prediction|    id|
+--------------------+----------+--------------------+----------+------+
|         Sorbus_Aria|       0.0|[0.32025702250384...|       0.0|1269.0|
|         Sorbus_Aria|       0.0|[0.27247980908622...|       0.0|1395.0|
|         Sorbus_Aria|       0.0|[0.26762364696292...|       0.0| 676.0|
|         Sorbus_Aria|       0.0|[0.26059767814940...|       0.0| 798.0|
|         Sorbus_Aria|       0.0|[0.27195400272790...|       0.0|1176.0|
|         Sorbus_Aria|       0.0|[0.24442803333658...|       0.0| 867.0|
|         Sorbus_Aria|       0.0|[0.32194640168715...|       0.0| 714.0|
|         Sorbus_Aria|       0.0|[0.28066739565063...|       0.0| 741.0|
|         Sorbus_Aria|       0.0|[0.31794566724273...|       0.0|1200.0|
|         Sorbus_Aria|       0.0|[0.35770505345511...|       0.0|1160.0|
|Betula_Austrosine...|       1.0|[2.89855072463768.

In [13]:
from pyspark.ml.feature import IndexToString
converter = IndexToString(inputCol="prediction", outputCol="predictedSpecies", labels=stIndModel.labels)
train_preds = converter.transform(train_preds)
train_preds.show()

+--------------------+----------+--------------------+----------+------+--------------------+
|             species|speciesEnc|         probability|prediction|    id|    predictedSpecies|
+--------------------+----------+--------------------+----------+------+--------------------+
|Magnolia_Salicifolia|      38.0|[0.00225803640496...|      38.0| 256.0|Magnolia_Salicifolia|
|Betula_Austrosine...|       1.0|[4.14078674948240...|       1.0| 488.0|Betula_Austrosine...|
|  Tilia_Platyphyllos|      87.0|[0.04214145977591...|      87.0| 239.0|  Tilia_Platyphyllos|
|     Ilex_Aquifolium|      22.0|[0.00884596880601...|      22.0| 600.0|     Ilex_Aquifolium|
|         Acer_Pictum|      82.0|[5.16099416355170...|      82.0|1417.0|         Acer_Pictum|
|  Quercus_Imbricaria|      31.0|[0.00168786949319...|      31.0| 497.0|  Quercus_Imbricaria|
|   Quercus_Agrifolia|      32.0|[0.0,0.0043520438...|      32.0| 387.0|   Quercus_Agrifolia|
|    Acer_Saccharinum|      79.0|[0.00309976178616...|      

In [21]:
from decimal import *
train_pred_rdd = train_preds.select('id', 'probability').rdd.map(lambda x: [int(x[0])] + [Decimal(a) for a in x[1]])
train_pred_df = spark.createDataFrame(train_pred_rdd, sol_schema).cache()

In [15]:
train_pred_df = train_pred_df.toPandas()

NameError: name 'train_pred_df' is not defined

In [ ]:
train_pred_df[train_pred_df.id == 488.0]

In [22]:
from pyspark.mllib.regression import IsotonicRegression, IsotonicRegressionModel
from pyspark.sql import SQLContext
tuple_schema = StructType([
    StructField("label", IntegerType(), True),
    StructField("prob", DecimalType(), True)])

def createTuple(x):
    l = 1 if int(x[0]) == int(x[1]) else 0
    return (int(l), x[2], 1)
create_tup_udf = spark.udf.register('createTuple', createTuple, tuple_schema)

r = train_preds.select('speciesEnc', 'probability').rdd.flatMap(lambda x: [ (x[0], i,v) for i,v in enumerate(x[1])])
r = r.map(createTuple)

In [23]:
r.take(40)

[(0, 0.0022580364049624262, 1),
 (0, 0.00031055900621118014, 1),
 (0, 0.0010979210106167206, 1),
 (0, 0.025422172265558873, 1),
 (0, 0.00056618768666760679, 1),
 (0, 0.017757206421942853, 1),
 (0, 0.00028824149630102264, 1),
 (0, 0.0026821921449109187, 1),
 (0, 0.020860990651688057, 1),
 (0, 0.13887524335730353, 1),
 (0, 0.00033628872221720657, 1),
 (0, 0.0, 1),
 (0, 0.001091582681752436, 1),
 (0, 0.00072082645893713616, 1),
 (0, 0.0069026535994297887, 1),
 (0, 0.00039621601527950467, 1),
 (0, 0.065386414249853017, 1),
 (0, 0.01674034295690878, 1),
 (0, 0.0018911318150448585, 1),
 (0, 0.021462056489179072, 1),
 (0, 0.0047086607713002172, 1),
 (0, 0.00024844720496894411, 1),
 (0, 0.0008091803141903486, 1),
 (0, 0.0023828237068874501, 1),
 (0, 0.00030521907759759422, 1),
 (0, 0.0075682644293074653, 1),
 (0, 0.0096708196982019309, 1),
 (0, 0.001013107148171594, 1),
 (0, 0.00092806398806123485, 1),
 (0, 0.011955368002624821, 1),
 (0, 0.005523176094991763, 1),
 (0, 0.0059294681436197613, 1)

In [ ]:
from pyspark.mllib.regression import IsotonicRegression, IsotonicRegressionModel
model = IsotonicRegression.train(r)




## Make predictions on the test set.

In [16]:
from pyspark.ml.classification import RandomForestClassificationModel
model = RandomForestClassificationModel.load('spark_random_forest_model_1')

In [17]:
prediction_df = model.transform(test_df_vec).cache()

In [18]:
species_list = stIndModel.labels
species_list.insert(0, 'id')

In [19]:
from decimal import *
pred_rdd = prediction_df.select('id', 'probability').rdd.map(lambda x: [int(x[0])] + [Decimal(a) for a in x[1]])

In [20]:
sol_fields = [StructField(field_name, DecimalType(38, 38), True) for field_name in species_list]
sol_fields[0].dataType = IntegerType()
sol_schema=StructType(sol_fields)    
pred_df = spark.createDataFrame(pred_rdd, sol_schema).cache()

In [60]:
import pandas as pd
pd_dataframe = pred_df.toPandas()
sorted_by_cols = pd_dataframe.sort_index(axis=1)
cols = list(sorted_by_cols.columns.values)
cols.remove('id')
cols.insert(0, 'id')
submission_df = sorted_by_cols[cols]


In [63]:
submission_df.head()

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,653,0.00094975172954717319590695057129892120,0.00012422360248447202719056237096850737,0.07354850125158279849912901227071415633,0.00366638601469200129875924965006106504,0E-38,0.00037251752681271963927797652438300702,0.00097081744910714514850552658842275378,0.00066465386883593784075191512883407086,0.00061492603081280739797664924850550960,...,0.02291310955464516407875663617232930847,0.00697124709934343091133346348442501039,0.00247000459012986861920269099357483356,0.00048656937856813832704930278261201693,0.00095015220030616011955698629876110317,0.00249220046718247713671279264247004903,0.00693460848809994423458391565873171203,0.01453479382853483520277038110180001240,0.00024316109422492396350407029004259130,0.00115329736531491716493291743717009012
1,1415,0.00276269219660280757056014522277109791,0.01641436807307629017205208299401419936,0.00121792059825917217685820048700406915,0.00536641267905056847109523943117892486,0.04464243943191310848295216828773845918,0.02825257815572187330976916541658283677,0.00191195688883015382092400713531787915,0.00123390174163540730471744222285224168,0.00519630738059833913872198607464270026,...,0.00090634194181104924849246007312331130,0.00471859819431322442817577922369309817,0.00193636051071293898741265682161838413,0.00356028012636329166049065086951941339,0.02182112789083639342591069976151629817,0.01188502964771783083908207601098183659,0.00172890981911444428613389145965584248,0.00241721708957016653829263930219894974,0.01575325144334485324981720566484000301,0.00424611300816044468647847409670248453
2,736,0.01221072511701751760193612739158197655,0.00603660660385448716303002925087639596,0.00172254341746276226632339501065871445,0.01479746426440261761192651590590685373,0.00472380052916299229132102155404027144,0.00115291343588312493462222363405089709,0.00161305214770173671080810695599438986,0.02513783689315314193168937606515100924,0.01960791658603302026198100804776913719,...,0.00090634194181104946533289457022419811,0.00514492367678587823875968609854680835,0.02699456816419191357292461930228455458,0.14091767748659017600587617380369920284,0.07391989127447813612281635187173378654,0.00804375661754638295042330042861067341,0.00499863280183367175713016195004456677,0.00385637072856403291185034554189314804,0E-38,0.01677942035628451553086470937614649301
3,1193,0.00951870171486804417249061316397273913,0.00176359558693875580724641505270255948,0.00186240355732290256002281481073623581,0.01751085136808004649733305768677382730,0.00030024349129050394375042842831646794,0.00213971660136967655255202380715218169,0.00310826236384218563754688879896548315,0.02036407217720742623967566942155826837,0.00764011533717242827257187798295490211,...,0.00137145822088081720431518295555406439,0.00733119522666879873395995304008465610,0.01761788813618097096491688091646210523,0.01938217519397773466738321701541281072,0.00724579428528890519267680048187685315,0.00852497771750523707090518854556648876,0.00481265239970905724792293867153603060,0.00517475646363766382529458809358402505,0E-38,0.01939630455298257255547511590521025937
4,1139,0.00449716559912244871755060415807747631,0.00139200062110370384980961855347914025,0.01150618607814925080379264699104169267,0.01322702124338554982574400753492227523,0.00030024349129050388954031980404124624,0.00228791946233849454936382095127100911,0.00259854967755216749447644275505808764,0.00250503598245698693228189490866952838,0.00370290086928260972487514379736239789,...,0.00252789487102696034184767803765225835,0.01517654653499122228976769832797799609,0.00392266311739860110141542293149541365,0.00572083381042039033903545686143843341,0.00527934344295909532829291777034086408,0.00930108280972844446432556964055038407,0.006172577813690832

Isotonic Regression
-figure out log loss.
-figure out isotonic regression.
-use log loss to see if isotonic regression is working
-Isotonic Regression takes tuples of values (label, feature, weight)
-label is equal to 1 or 0, based on if is is the predicted label.
-run isotonic regression. then split it up again.



In [64]:
submission_df.to_csv('spark_sol_2.csv', index=False)